## Code Aim
Data was given in a sequence of bouts of different durations. The goal was to create a sequence of 5 sec long bouts of
bouts to later pass this sequence to R Markov Package to fit the sequence and compute the probability transition matrix

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import glob
import os

In [2]:
os.chdir('/Users/bmr225/Documents/PythonCodes/Survival_Data')

In [3]:
phenotype = ['Susceptible', 'Resilient', 'Control']
phase = ['Light','Dark']
condition = ['pre','post']

# Loading the data
def load_data(phenotype,phase,condition):
    df = pd.read_csv(phenotype + "_"+ phase + "_" + condition + "_grouped.csv")
    return df

In [ ]:
for p in phenotype:
    for ph in phase:
        for c in condition:

            df = load_data(p,ph,c)   


            #Grouping by Mouse ID
            df_grouped_dur = df.groupby('Mouse ID')['Duration'].apply(list)
            df_grouped_state = df.groupby('Mouse ID')['State Number'].apply(list)

            # Lists length
            df_grouped_len =  df.groupby('Mouse ID')['Duration'].count()


            state_dict = {}
            
            for i in range(len(df_grouped_dur)):
                
                #l_dur = [df_grouped_dur.iloc[i] for i in range(len(df_grouped_dur))]
            
                l_dur = df_grouped_dur.iloc[i] 
                l_state = df_grouped_state.iloc[i] 
    
                 # making the shortest bout duration 5
                l_dur =[5 if x<5 else x for x in l_dur]
    
    
                new_l = []
    
                for j in range(len(l_dur)): 
                    repeats = l_dur[j]//5
                    repeat_l = [l_state[j]]*repeats
                    new_l.append(repeat_l)
     
    
                new_l= sum(new_l,[]) 
    
                # Need to pad this list with NA based on the longest list in df_grouped_len
   
                state_dict.update({i:new_l})

                # To generate a dataframe of different lengths of observations for each column
                state_seq_df = pd.DataFrame.from_dict(state_dict, orient='index').T

            

                # Changing the number to state
                ###############################
                labels={1:'Wake',2:'NREM', 4:'REM'}
                for col in state_seq_df.columns:
                    state_seq_df[col] = state_seq_df[col].map(labels)
               
                filename = 'Markov_seq_'+ p + c +  ph +'.csv'
                state_seq_df.to_csv(filename,index = False)
